In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

# Load pre-trained InceptionV3 (without the classification layer)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Load CIFAR10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocess CIFAR10 images
x_train = tf.image.resize(x_train, (299, 299)).numpy() # Resize to match InceptionV3 input
x_test = tf.image.resize(x_test, (299, 299)).numpy()
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

In [ ]:
# Add a new classification layer for CIFAR10
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10, activation='softmax') # 10 classes for CIFAR10
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Evaluate the model on CIFAR10
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Initial Accuracy: {accuracy:.4f}")
initial_loss=loss
initial_accuracy=accuracy

In [ ]:
# Fine-tune some layers of the base model
for layer in base_model.layers[-10:]:  # Fine-tune the last 10 layers
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower learning rate
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=8),
          epochs=10,  # Adjust as needed
          validation_data=(x_test, y_test))

# Evaluate the fine-tuned model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Fine-tuned Accuracy: {accuracy:.4f}")

ft_loss=loss
ft_accuracy=accuracy

In [ ]:
import pandas as pd

data = {
    'Model': ['before fine-tuning', 'after fine-tuning'],
    'Accuracy': [initial_accuracy,ft_accuracy],
}

df_comparison = pd.DataFrame(data)
print(df_comparison)

In [2]:
del base_model
#del x_train, y_train
#del x_test, y_test
import gc; gc.collect()

NameError: name 'base_model' is not defined